In [1]:
import gym
from gym import wrappers
from gym.spaces.utils import flatdim

import torch
from torch import nn
from torch.functional import F
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import cv2
from tqdm import tqdm

from copy import deepcopy
import mediapy
import collections

# Comment out for debugging
import warnings
warnings.filterwarnings('ignore')

/home/luke/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


class SequenceModel(nn.Module):
    pass

class Encoder(nn.Module):
    pass

class DynamicsModel(nn.Module):
    pass

class RewardPredictor(nn.Module):
    pass

class TerminationPredictor(nn.Module): # called "continue predictor"
    pass

class Decoder(nn.Module):
    pass